# Validação Cruzada

## Descrição

Use os dados do [Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra) UCI Machine Learning Repository.  

Discuta se é preciso normalizar os dados ou não. Se for preciso, faça a normalizaçao.  
 
Você vai verificar qual dentre os algoritmos abaixo é o melhor para classificar os dados. Utilize 20% dos dados para teste estratificando pela classe.

- Decision Tree  (criterion{“gini”, “entropy”, “log_loss”}, max_depth de 10 a 100 em multiplos de 10)
- K vizinhos (K = 1,3,5,11,21,31)  

Verifique que algoritmo tem a melhor acuracia media. Vamos usar o desenho experimental discutido em classe, de um k-fold para medir a acuracia media e um k-fold interno para escolher os hiperparâmetros (para cada conjunto de folds de treino). O algoritmo experimental esta descrito abaixo. Use 5-fold tanto para a avaliacao externa (para calcular a acuracia media) quanto para o loop interno (para escolher os hiperparametros).  

Obtenha o melhor conjunto de hiperparametros, retreine o modelo usando todos os dados de treino e avalie o desempenho com os dados de teste.

## Preparando os dados

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
dados = pd.read_csv('dataR2.csv')

dados

X = dados.iloc[:, : -1]
y = dados['Classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

normalizacao = StandardScaler()
normalizacao.fit(X_train)

X_train_std = pd.DataFrame(normalizacao.transform(X_train), columns=X_train.columns.tolist())
X_test_std = pd.DataFrame(normalizacao.transform(X_test), columns=X_train.columns.tolist())

## Decision Tree

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#### Descobrindo e armazenando os melhores parâmetros para Árvore de Decisão

In [4]:
criterion = ['gini', 'entropy', 'log_loss']
maxDepth = range(10, 101, 10)

clf_cv = DecisionTreeClassifier(random_state=42)
param_grid = {"criterion": np.array(criterion), "max_depth": np.array(maxDepth)}
clf_gscv = GridSearchCV(clf_cv, param_grid, cv=5)

# fit nos dados de treino
clf_gscv.fit(X_train_std, y_train)

# Checa quais sãos os parâmetros que geram a melhor acurácia média
bestCriterion, bestMaxDepth = clf_gscv.best_params_.values()
print(bestCriterion, bestMaxDepth)

entropy 10


#### Testando os dados de teste com os melhores parâmetros

In [5]:
y_pred = clf_gscv.predict(X_test_std)
score = accuracy_score(y_test, y_pred)

print("Acurácia nos dados de teste: {0:.2f}%".format(np.mean(score) * 100))


Acurácia nos dados de teste: 75.00%


## KNeighborsClassifier (KNN)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#### Descobrindo e armazenando os melhores parâmetros para o KNN

In [7]:
kVizinhos = [1, 3, 5, 11, 21, 31]

knn_cv = KNeighborsClassifier()
param_grid = {"n_neighbors": np.array(kVizinhos)}
knn_gscv = GridSearchCV(knn_cv, param_grid, cv=5)

# fit nos dados de treino
knn_gscv.fit(X_train_std, y_train)

# Checa quais sãos os parâmetros que geram a melhor acurácia média
bestN = knn_gscv.best_params_["n_neighbors"]
print(bestN)

21


#### Testando os dados de teste com os melhores parâmetros

In [8]:
y_pred = knn_gscv.predict(X_test_std)
score = accuracy_score(y_test, y_pred)

print("Acurácia nos dados de teste: {0:.2f}%".format(np.mean(score) * 100))

Acurácia nos dados de teste: 66.67%


## Comparação dos resultados

|Modelo|Acurácia nos dados de teste (melhores hiperparâmetros)|Hiperparâmetros|
|------|-----------------------------------|---------------|
|Decision Tree|75.00%|Criterion: entropy; Max-depth: 10|
|KNN|66.67%|K-vizinhos: 21|